# Calibrate the rtl-sdr dongle

In [1]:
library(tidyverse)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
library(yaml)

In [3]:
paste("Running at ", Sys.time() %>% .POSIXct("GMT"), "GMT")

[1] "Running at  2019-09-28 08:57:44 GMT"

Use `kal` to find a nearby GSM station in the GSM900 band with a default offset of 0 and gain of 49.6

In [4]:
best_channel <- system(paste0("kal -s GSM900 -e 0 -g 49.6 | sed 's/ \\+/\\t/g' | cut -f 3,8 | ",
                              "sort -k2 -n -r | head -1 | cut -f 1"),
                       intern = TRUE)

Use that channel to find the clock frequency offset

In [64]:
(clock_offset <- system(paste0("kal -c ", best_channel, " -g 49.6 -e 0"), intern=TRUE) %>%
    pluck(5) %>% strsplit(":") %>% pluck(1,2) %>% substr(2,nchar(.) - 4) %>% as.numeric)

[1] -0.605

In [98]:
write_yaml(list("clock_offset" = clock_offset), "calibration.yaml")